In [1]:
import tdd

In [2]:
tdd.__version__

'0.0.6dev11'

In [3]:
import os
import pathlib

In [4]:
from tdd import read_simsurvey, select_using_simsurvey_meta

In [5]:
import numpy as np
import pandas as pd

##  Location of example datasets

In [6]:
from tdd import example_data

In [7]:
lcs_fname  = os.path.join(example_data, 'salt2_ex_lcs.pkl')
params_fname = os.path.join(example_data, 'salt2_ex_params.csv')

In [8]:
params_fname

'/Users/rbiswas/soft/mypython3/lib/python3.7/site-packages/tdd-0.0.6.dev11-py3.7.egg/tdd/example_data/salt2_ex_params.csv'

In [9]:
# read in params, and replace index name
params = pd.read_csv(params_fname)
params.rename(columns=dict(idx='tid'), inplace=True)
params.set_index('tid', inplace=True)

This is the set of parameters that were used for the simulation.

In [10]:
params.head()

,z,x0,mBstar,x1,c,Mabs,MnoDisp
tid,,,,,,,
0,0.030175,0.001727,17.410036,-3.982571,0.088835,-18.267305,-18.463587
1,0.171198,0.000109,20.412162,-0.228617,0.140155,-19.237994,-18.828046
2,0.097653,0.000619,18.514598,3.331037,0.093523,-19.815452,-19.472775
3,0.159323,0.000347,19.146437,-1.551270,-0.327156,-20.331998,-20.109765
4,0.176668,0.000127,20.245215,-3.082114,-0.102774,-19.480330,-19.191111


In [11]:
import time

In [12]:
def phot_selection(meta, phot, lcs, num_points=7):
    x = phot.query('SNR > 5').groupby('tid').agg(dict(band='count'))
    selected = x.query('band >= @num_points').index.values
    meta = meta.loc[selected]
    query = 'tid in ("'+'", "'.join(list(x.index.values)) +'")'
    phot  = phot.query(query)
    return meta, phot

### First read in all of the simulaton

In [13]:
ts = time.time()
meta_all, phot_all = read_simsurvey(lcs_fname, shift_idx=0, sim_suffix=0,
                            meta_selection_func=None,
                            params=params, phot_selection=None)
te = time.time()

In [14]:
meta_all

,z,t0,mwr_v,x0,x1,c,mwebv,ra,dec,mwebv_sfd98,order,mBstar,Mabs,MnoDisp
tid,,,,,,,,,,,,,,
0_0,0.030175,2.458482e+06,3.1,0.001727,-3.982571,0.088835,0.007585,211.232889,24.535223,0.016379,0,17.410036,-18.267305,-18.463587
3_0,0.159323,2.458465e+06,3.1,0.000347,-1.551270,-0.327156,0.005076,165.981738,41.498977,0.007253,1,19.146437,-20.331998,-20.109765
9_0,0.022406,2.458480e+06,3.1,0.013854,-3.448262,-0.261963,0.013789,147.234026,10.092918,0.025787,2,15.146525,-19.871907,-19.639547
13_0,0.002627,2.458361e+06,3.1,0.449468,2.227740,0.246635,0.002670,353.508998,-25.057846,0.017560,3,11.364438,-18.967488,-18.837697
18_0,0.020180,2.458477e+06,3.1,0.039859,-1.929127,-0.393876,0.087138,297.453874,58.834121,0.089289,4,13.994212,-20.793462,-20.266299
22_0,0.156265,2.458397e+06,3.1,0.000237,2.856853,0.103387,0.011222,133.474869,39.192337,0.025572,5,19.559388,-19.872904,-19.375428
23_0,0.108513,2.458442e+06,3.1,0.000461,-1.272054,-0.206786,0.068643,329.235645,15.931547,0.058604,6,18.841761,-19.732848,-19.771015
45_0,0.003373,2.458427e+06,3.1,0.876478,1.513916,-0.045681,0.015394,350.183559,-10.352431,0.023620,7,10.643761,-20.231859,-19.655342


### Do only a selection based on metadata. This is the fastest.

In [15]:
ts = time.time()
meta_msel, phot_msel = read_simsurvey(lcs_fname, shift_idx=0, sim_suffix=0,
                            meta_selection_func=select_using_simsurvey_meta,
                            params=params)
te = time.time()

In [16]:
meta_msel

,z,t0,mwr_v,x0,x1,c,mwebv,ra,dec,mwebv_sfd98,order,mBstar,Mabs,MnoDisp
tid,,,,,,,,,,,,,,
0_0,0.030175,2.458482e+06,3.1,0.001727,-3.982571,0.088835,0.007585,211.232889,24.535223,0.016379,0,17.410036,-18.267305,-18.463587
9_0,0.022406,2.458480e+06,3.1,0.013854,-3.448262,-0.261963,0.013789,147.234026,10.092918,0.025787,2,15.146525,-19.871907,-19.639547
13_0,0.002627,2.458361e+06,3.1,0.449468,2.227740,0.246635,0.002670,353.508998,-25.057846,0.017560,3,11.364438,-18.967488,-18.837697
18_0,0.020180,2.458477e+06,3.1,0.039859,-1.929127,-0.393876,0.087138,297.453874,58.834121,0.089289,4,13.994212,-20.793462,-20.266299
45_0,0.003373,2.458427e+06,3.1,0.876478,1.513916,-0.045681,0.015394,350.183559,-10.352431,0.023620,7,10.643761,-20.231859,-19.655342


### Selection on the basis of both metadata and light curve properties

In [17]:
ts = time.time()
meta_mpsel, phot_mpsel = read_simsurvey(lcs_fname, shift_idx=0, sim_suffix=0,
                            meta_selection_func=select_using_simsurvey_meta,
                            params=params, phot_selection=phot_selection)
te = time.time()

In [18]:
meta_mpsel

,z,t0,mwr_v,x0,x1,c,mwebv,ra,dec,mwebv_sfd98,order,mBstar,Mabs,MnoDisp
tid,,,,,,,,,,,,,,
13_0,0.002627,2.458361e+06,3.1,0.449468,2.227740,0.246635,0.002670,353.508998,-25.057846,0.017560,3,11.364438,-18.967488,-18.837697
45_0,0.003373,2.458427e+06,3.1,0.876478,1.513916,-0.045681,0.015394,350.183559,-10.352431,0.023620,7,10.643761,-20.231859,-19.655342
9_0,0.022406,2.458480e+06,3.1,0.013854,-3.448262,-0.261963,0.013789,147.234026,10.092918,0.025787,2,15.146525,-19.871907,-19.639547
